# Lifetime Fuel Cost Savings (LFCS) Analysis  
The LFCS describes the total discounted fuel cost savings for an EV when compared to a conventional vehicle over a fixed lifespan and identical operating conditions (i.e., annual miles driven). It is an aggregate measure of the discounted cumulative cost savings associated with lower present day and projected future fuel costs for EVs. From a cost of ownership perspective, the LFCS offsets the purchase price premium paid by EV owners.

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# discount rate
dr = 0.035 #source: Mercatus Center
dr_low = 0.03
dr_high = 0.07

# powertrain efficiencies
icev_eff = 29 #mpg (source: EPA)
bev_eff = 119 #mpgge (source: EPA)
hev_eff = 45 #mpg (source: EPA)

# lifetime vmt
vmt = np.array([13065] * 3 + [12582] * 3 + [11432] * 4 + [7812] * 5) #source: NHTS 2017 (Table 22)
vmt_low = vmt * 100e3 / vmt.sum() #100k lifetime VMT
vmt_high = vmt * 200e3 / vmt.sum() #200k lifetime VMT

kwh_to_gge = 1 / 33.7 #source: https://epact.energy.gov/fuel-conversion-factors
bev_kwh_per_mile = 1 / (bev_eff * kwh_to_gge)
phev_chevy_volt_pct_e_miles = 0.76 #source: 2018 EPA Automotive Trends Report (Cheverolet Volt)
phev_prius_prime_e_miles = 0.53 #source 2018 EPA Automotive Trends Report (Prius Prime w/ 25 miles e-range)
lifespan = 15

# load data
aaa_df = pd.read_csv('../data/aaa/190702_fuel_prices_aaa.csv')
eia_df = pd.read_csv('../data/eia/15yr-gas-electricity-price-projections/eia_aeo19_gas_electricity_price_projections.csv', skiprows=4)

bev_elec_baseline_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_baseline.csv')
bev_elec_lower_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_lower_bnd.csv')
bev_elec_lower_df.rename(columns={'lcoc_min': 'lcoc_cost_per_kwh'}, inplace=True)
bev_elec_upper_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_upper_bnd.csv')
bev_elec_upper_df.rename(columns={'lcoc_max': 'lcoc_cost_per_kwh'}, inplace=True)

phev_elec_baseline_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_phev_baseline.csv')
phev_elec_lower_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_phev_lower_bnd.csv')
phev_elec_lower_df.rename(columns={'lcoc_min': 'lcoc_cost_per_kwh'}, inplace=True)
phev_elec_upper_df = pd.read_csv('../outputs/cost-of-charging/comb/comb_states_phev_upper_bnd.csv')
phev_elec_upper_df.rename(columns={'lcoc_max': 'lcoc_cost_per_kwh'}, inplace=True)

In [3]:
# Preprocessing
eia_df.rename(columns={'Residential: Electricity: United States: Reference case Indexed to 2019 as percent': 'ref_elec_cost_sf',
                       'Residential: Electricity: United States: High oil price Indexed to 2019 as percent': 'highoil_elec_cost_sf',
                       'Residential: Electricity: United States: Low oil price Indexed to 2019 as percent': 'lowoil_elec_cost_sf',
                       'Transportation: Motor Gasoline: United States: Reference case Indexed to 2019 as percent': 'ref_gas_cost_sf',
                       'Transportation: Motor Gasoline: United States: High oil price Indexed to 2019 as percent': 'highoil_gas_cost_sf',
                       'Transportation: Motor Gasoline: United States: Low oil price Indexed to 2019 as percent': 'lowoil_gas_cost_sf'},
             inplace=True)

eia_df = eia_df[(eia_df.Year >= 2019)&(eia_df.Year < 2034)].sort_values('Year').reset_index(drop=True)
eia_df['ref_elec_cost_sf'] = 1 + eia_df['ref_elec_cost_sf'] / 100
eia_df['highoil_elec_cost_sf'] = 1 + eia_df['highoil_elec_cost_sf'] / 100
eia_df['lowoil_elec_cost_sf'] = 1 + eia_df['lowoil_elec_cost_sf'] / 100
eia_df['ref_gas_cost_sf'] = 1 + eia_df['ref_gas_cost_sf'] / 100
eia_df['highoil_gas_cost_sf'] = 1 + eia_df['highoil_gas_cost_sf'] / 100
eia_df['lowoil_gas_cost_sf'] = 1 + eia_df['lowoil_gas_cost_sf'] / 100

aaa_df = aaa_df.sort_values('state').reset_index(drop=True)

bev_elec_baseline_df = bev_elec_baseline_df.sort_values('state').reset_index(drop=True)
bev_elec_lower_df = bev_elec_lower_df.sort_values('state').reset_index(drop=True)
bev_elec_upper_df = bev_elec_upper_df.sort_values('state').reset_index(drop=True)

phev_elec_baseline_df = phev_elec_baseline_df.sort_values('state').reset_index(drop=True)
phev_elec_lower_df = phev_elec_lower_df.sort_values('state').reset_index(drop=True)
phev_elec_upper_df = phev_elec_upper_df.sort_values('state').reset_index(drop=True)

In [4]:
# Calculate lifetime fuel costs 

### options ###
tot_miles = vmt #{vmt, vmt_low, vmt_high}
elec_price_scen = 'ref_elec_cost_sf' #{'ref_elec_cost_sf', 'highoil_elec_cost_sf', 'lowoil_elec_cost_sf'}
gas_price_scen = 'ref_gas_cost_sf' #{'ref_gas_cost_sf', 'highoil_gas_cost_sf', 'lowoil_gas_cost_sf'}
discount_rate = dr #{dr, dr_low, dr_high}
phev_utility_factor = phev_chevy_volt_pct_e_miles #{phev_chevy_volt_pct_e_miles, phev_prius_prime_e_miles}
### ~~~~~~ ###

# gasoline for ICEV, PHEV
icev_gallons = tot_miles / icev_eff
phev_gallons = (tot_miles * (1 - phev_utility_factor)) / hev_eff

for y, l in zip(range(2019, 2019 + lifespan), range(lifespan)):
    colname = str(y)
    cost_wo_discount = aaa_df['regular'] * float(eia_df[eia_df.Year==y][gas_price_scen])
    cost_w_discount = cost_wo_discount / (1 + discount_rate)**l
    aaa_df[colname] = cost_w_discount
    
# electricity for BEV, PHEV
dfs = [bev_elec_baseline_df, bev_elec_lower_df, bev_elec_upper_df,
       phev_elec_baseline_df, phev_elec_lower_df, phev_elec_upper_df]

bev_kwh = tot_miles * bev_kwh_per_mile
phev_kwh = tot_miles * phev_utility_factor * bev_kwh_per_mile

for df in dfs:
    for y, l in zip(range(2019, 2019 + lifespan), range(lifespan)):
        colname = str(y)
        cost_wo_discount = df['lcoc_cost_per_kwh'] * float(eia_df[eia_df.Year==y][elec_price_scen])
        cost_w_discount = cost_wo_discount / (1 + discount_rate)**l
        df[colname] = cost_w_discount
    
icev_lifetime_fuel_cost = (aaa_df.loc[:, '2019':] * icev_gallons).sum(axis=1)
phev_lifetime_fuel_cost_gas = (aaa_df.loc[:, '2019':] * phev_gallons).sum(axis=1)
bev_baseline_lifetime_fuel_cost = (bev_elec_baseline_df.loc[:, '2019':] * bev_kwh).sum(axis=1)
bev_lower_lifetime_fuel_cost = (bev_elec_lower_df.loc[:, '2019':] * bev_kwh).sum(axis=1)
bev_upper_lifetime_fuel_cost = (bev_elec_upper_df.loc[:, '2019':] * bev_kwh).sum(axis=1)
phev_baseline_lifetime_fuel_cost_elec = (phev_elec_baseline_df.loc[:, '2019':] * phev_kwh).sum(axis=1)
phev_lower_lifetime_fuel_cost_elec = (phev_elec_lower_df.loc[:, '2019':] * phev_kwh).sum(axis=1)
phev_upper_lifetime_fuel_cost_elec = (phev_elec_upper_df.loc[:, '2019':] * phev_kwh).sum(axis=1)

phev_baseline_lifetime_fuel_cost = phev_lifetime_fuel_cost_gas + phev_baseline_lifetime_fuel_cost_elec
phev_lower_lifetime_fuel_cost = phev_lifetime_fuel_cost_gas + phev_lower_lifetime_fuel_cost_elec
phev_upper_lifetime_fuel_cost = phev_lifetime_fuel_cost_gas + phev_upper_lifetime_fuel_cost_elec

In [5]:
# Create lifetime fuel costs DataFrame
states = aaa_df['state']
lfc_df = pd.DataFrame({'state': states,
                       'lfc_icev': icev_lifetime_fuel_cost,
                       'lfc_bev_baseline': bev_baseline_lifetime_fuel_cost,
                       'lfc_bev_lower': bev_lower_lifetime_fuel_cost,
                       'lfc_bev_upper': bev_upper_lifetime_fuel_cost,
                       'lfc_phev_baseline': phev_baseline_lifetime_fuel_cost,
                       'lfc_phev_lower': phev_lower_lifetime_fuel_cost,
                       'lfc_phev_upper': phev_upper_lifetime_fuel_cost
                      })

lfc_df['lfcs_bev_baseline'] = lfc_df['lfc_icev'] - lfc_df['lfc_bev_baseline']
lfc_df['lfcs_bev_lower'] = lfc_df['lfc_icev'] - lfc_df['lfc_bev_lower']
lfc_df['lfcs_bev_upper'] = lfc_df['lfc_icev'] - lfc_df['lfc_bev_upper']
lfc_df['lfcs_phev_baseline'] = lfc_df['lfc_icev'] - lfc_df['lfc_phev_baseline']
lfc_df['lfcs_phev_lower'] = lfc_df['lfc_icev'] - lfc_df['lfc_phev_lower']
lfc_df['lfcs_phev_upper'] = lfc_df['lfc_icev'] - lfc_df['lfc_phev_upper']

In [7]:
lfc_df.sort_values('state').to_csv('../outputs/lfcs-analysis/state_results.csv', index=False)